<a href="https://colab.research.google.com/github/arnabbiswas66/multimodal-fake-news-classifier/blob/main/Twitter_multimodal_classifier_Scaled_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
#!pip install -q -U "tensorflow-text==2.11.*"
!pip install -q tensorflow_text
!pip install -q talos

In [1]:
from tensorflow.python.platform.tf_logging import warn
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import re
from os import listdir
import shutil
import glob
import random
import matplotlib.pyplot as plt
import talos as ta
import tensorflow_hub as hub
import tensorflow_text as text
tf.get_logger().setLevel('ERROR')
import warnings

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
BASE_SAVE_LOCATION = "/content/drive/MyDrive/multimodal-news"
CHECKPOINT_FILEPATH = '/content/drive/MyDrive/multimodal-news/twitter-models/'

## Load the dataframes

In [3]:
#Load training df
train_df_model = pd.read_pickle(BASE_SAVE_LOCATION+"/twitter-train.pickle")
test_df_model = pd.read_pickle(BASE_SAVE_LOCATION+"/twitter-test.pickle")
print(train_df_model.shape, test_df_model.shape)

(14258, 3) (1923, 3)


In [4]:
# This is for Talos as it doesn't take TF DataSet as input
X_train = train_df_model.copy()
y_train = X_train.pop('label')

X_test = test_df_model.copy()
y_test = X_test.pop('label')

# Data input pipeline

In [5]:
# Define TF Hub paths to the BERT encoder and its preprocessor
bert_model_path = (
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1"
)
bert_preprocess_path = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [6]:
def make_bert_preprocessing_model(sentence_features, seq_length=224):
    """Returns Model mapping string features to BERT inputs.

  Args:
    sentence_features: A list with the names of string-valued features.
    seq_length: An integer that defines the sequence length of BERT inputs.

  Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
  """

    input_segments = [
        tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
        for ft in sentence_features
    ]

    # Tokenize the text to word pieces.
    bert_preprocess = hub.load(bert_preprocess_path)
    tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name="tokenizer")
    segments = [tokenizer(s) for s in input_segments]

    # Optional: Trim segments in a smart way to fit seq_length.
    # Simple cases (like this example) can skip this step and let
    # the next step apply a default truncation to approximately equal lengths.
    truncated_segments = segments

    # Pack inputs. The details (start/end token ids, dict of output tensors)
    # are model-dependent, so this gets loaded from the SavedModel.
    packer = hub.KerasLayer(
        bert_preprocess.bert_pack_inputs,
        arguments=dict(seq_length=seq_length),
        name="packer",
    )
    model_inputs = packer(truncated_segments)
    return keras.Model(input_segments, model_inputs)


In [7]:
bert_preprocess_model = make_bert_preprocessing_model(["text_1"])
bert_preprocess_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_1 (InputLayer)         [(None,)]                 0         
                                                                 
 tokenizer (KerasLayer)      (None, None, None)        0         
                                                                 
 packer (KerasLayer)         {'input_mask': (None, 22  0         
                             4),                                 
                              'input_word_ids': (None            
                             , 224),                             
                              'input_type_ids': (None            
                             , 224)}                             
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [8]:
idx = np.random.choice(len(train_df_model))
row = train_df_model.iloc[idx]
sample_text_1, sample_text_2 = row["tweetText"], row["tweetText"]
print(f"Text 1: {sample_text_1}")
print(f"Text 2: {sample_text_2}")

test_text = [np.array([sample_text_1])]
text_preprocessed = bert_preprocess_model(test_text)

print("Keys           : ", list(text_preprocessed.keys()))
print("Shape Word Ids : ", text_preprocessed["input_word_ids"].shape)
print("Word Ids       : ", text_preprocessed["input_word_ids"][0, :16])
print("Shape Mask     : ", text_preprocessed["input_mask"].shape)
print("Input Mask     : ", text_preprocessed["input_mask"][0, :16])
print("Shape Type Ids : ", text_preprocessed["input_type_ids"].shape)
print("Type Ids       : ", text_preprocessed["input_type_ids"][0, :16])


Text 1: #SANDY ~ Prayers go out to the people affected in Hoboken, NJ http://t.co/Ax9qluv5
Text 2: #SANDY ~ Prayers go out to the people affected in Hoboken, NJ http://t.co/Ax9qluv5
Keys           :  ['input_mask', 'input_word_ids', 'input_type_ids']
Shape Word Ids :  (1, 224)
Word Ids       :  tf.Tensor(
[  101  1001  7525  1066 12583  2175  2041  2000  1996  2111  5360  1999
  7570  5092  7520  1010], shape=(16,), dtype=int32)
Shape Mask     :  (1, 224)
Input Mask     :  tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(16,), dtype=int32)
Shape Type Ids :  (1, 224)
Type Ids       :  tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(16,), dtype=int32)


In [9]:
def dataframe_to_dataset(dataframe):
    columns = ['tweetText', 'image_1', 'label']
    dataframe = dataframe[columns].copy()
    labels = dataframe.pop("label")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

## Preprocessing utilities

In [10]:
resize = (224, 224)
bert_input_features = ["input_word_ids", "input_type_ids", "input_mask"]

In [11]:
def preprocess_image(image_path):
  extension = tf.strings.split(image_path,'.')[-1]
  image = tf.io.read_file(image_path)
  if extension == b"gif":
    image = tf.io.decode_image(image, 3, expand_animations=False)
  elif extension == b"png":
    image = tf.image.decode_png(image, 3)
  else:
    image = tf.image.decode_jpeg(image, 3)
  image = tf.image.resize(image, resize)
  return image

In [12]:
def preprocess_text(text_1):
  text_1 = tf.convert_to_tensor([text_1])
  output = bert_preprocess_model([text_1])
  output = {feature: tf.squeeze(output[feature]) for feature in bert_input_features}
  return output

In [13]:
def preprocess_text_and_image(sample):
  image_1 = preprocess_image(sample["image_1"])
  text = preprocess_text(sample["tweetText"])
  return {"image_1": image_1, "text": text}

In [14]:
batch_size = 32
auto = tf.data.AUTOTUNE

In [15]:
def prepare_dataset(dataframe, training=True):
  ds = dataframe_to_dataset(dataframe)
  if training:
      ds = ds.shuffle(len(train_df_model))
  ds = ds.map(lambda x, y: (preprocess_text_and_image(x), y)).cache()
  ds = ds.batch(batch_size).prefetch(auto)
  return ds

In [16]:
train_ds = prepare_dataset(train_df_model)
#validation_ds = prepare_dataset(val_df_model, False)
test_ds = prepare_dataset(test_df_model, False)

## Model Building

### Projection

In [17]:
def project_embeddings(
    embeddings, num_projection_layers, projection_dims, dropout_rate
):
    projected_embeddings = keras.layers.Dense(units=projection_dims)(embeddings)
    for _ in range(num_projection_layers):
        x = tf.nn.gelu(projected_embeddings)
        x = keras.layers.Dense(projection_dims)(x)
        x = keras.layers.Dropout(dropout_rate)(x)
        x = keras.layers.Add()([projected_embeddings, x])
        projected_embeddings = keras.layers.LayerNormalization()(x)
    return projected_embeddings


### Vision encoder

In [18]:
def create_vision_encoder(
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    # Load the pre-trained ResNet50V2 model to be used as the base encoder.
    resnet_v2 = keras.applications.EfficientNetV2B3(
        include_top=False, weights="imagenet", pooling="avg"
    )
    # Set the trainability of the base encoder.
    for layer in resnet_v2.layers:
        layer.trainable = trainable

    # Receive the images as inputs.
    image_1 = keras.Input(shape=(224, 224, 3), name="image_1")
    
    # Preprocess the input image.
    preprocessed_1 = keras.applications.resnet_v2.preprocess_input(image_1)
    
    # Generate the embeddings for the images using the resnet_v2 model
    # concatenate them.
    embeddings = resnet_v2(preprocessed_1)
    #embeddings = keras.layers.Concatenate()([embeddings_1, embeddings_2])

    # Project the embeddings produced by the model.
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    # Create the vision encoder model.
    return keras.Model([image_1], outputs, name="vision_encoder")

### Text Encoder

In [19]:
def create_text_encoder(
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    # Load the pre-trained BERT model to be used as the base encoder.
    bert = hub.KerasLayer(bert_model_path, name="bert",)
    # Set the trainability of the base encoder.
    bert.trainable = trainable

    # Receive the text as inputs.
    bert_input_features = ["input_type_ids", "input_mask", "input_word_ids"]
    inputs = {
        feature: keras.Input(shape=(224,), dtype=tf.int32, name=feature)
        for feature in bert_input_features
    }

    # Generate embeddings for the preprocessed text using the BERT model.
    embeddings = bert(inputs)["pooled_output"]

    # Project the embeddings produced by the model.
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    # Create the text encoder model.
    return keras.Model(inputs, outputs, name="text_encoder")

### Multi Head Attention Layer

In [20]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, x, y, training):
        attn_output = self.att(x, y)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

## MultiModal model

In [21]:
def create_multimodal_model(
    num_projection_layers=0,
    projection_dims=224,
    dropout_rate=0.1,
    vision_trainable=False,
    text_trainable=False,
    attention=False
):
    # Receive the images as inputs.
    image_1 = keras.Input(shape=(224, 224, 3), name="image_1")
    
    # Receive the text as inputs.
    bert_input_features = ["input_type_ids", "input_mask", "input_word_ids"]
    text_inputs = {
        feature: keras.Input(shape=(224,), dtype=tf.int32, name=feature)
        for feature in bert_input_features
    }

    # Create the encoders.
    vision_encoder = create_vision_encoder(
        num_projection_layers, projection_dims, dropout_rate, vision_trainable
    )
    text_encoder = create_text_encoder(
        num_projection_layers, projection_dims, dropout_rate, text_trainable
    )

    # Fetch the embedding projections.
    vision_projections = vision_encoder([image_1])
    vision_projections = keras.layers.Dropout(dropout_rate)(vision_projections)
    text_projections = text_encoder(text_inputs)
    text_projections = keras.layers.Dropout(dropout_rate)(text_projections)
    
    # Cross-attention.
    if attention:
      transformer_block = TransformerBlock(projection_dims, 4, projection_dims)
      x = transformer_block(tf.expand_dims(vision_projections, -1), tf.expand_dims(text_projections, -1))
      x = tf.keras.layers.Flatten()(x)
      
    # Concatenate the projections and pass through the classification layer.
    concatenated = keras.layers.Concatenate()([vision_projections, text_projections])
    if attention:
        concatenated = keras.layers.Concatenate()([concatenated, x])
        #x = tf.keras.layers.Flatten(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(concatenated)
    return keras.Model([image_1, text_inputs], outputs)

In [22]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics= [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

## Final Multimodal model

In [23]:
def multimodal_model(X_train, y_train, X_test, y_test, params):
  train_ds = prepare_dataset(train_df_model)
  test_ds = prepare_dataset(test_df_model, False)
  model = create_multimodal_model(params['num_projection_layers'],
    params['projection_dims'],
    params['dropout_rate'],
    params['vision_trainable'],
    params['text_trainable'],
    params['attention'])
  model.compile(
    optimizer=tf.keras.optimizers.Adam(params['lr']), loss=loss, metrics=metrics
  )
  history = model.fit(
      train_ds, validation_data=test_ds, 
      epochs=params['epochs'], batch_size=params['batch_size'])
  return history, model
  

In [ ]:
params = {
    'num_projection_layers' : [0],
    'projection_dims' : [128, 224],
    'dropout_rate' : [0.1, 0.2],
    'vision_trainable' : [False],
    'text_trainable' : [False],
    'attention' : [True],
    'lr' : [0.001, 0.0005],
    'epochs' : [10],
    'batch_size' : [32, 64]
}

In [ ]:
h = ta.Scan(x = X_train, y= y_train, params = params, model = multimodal_model, x_val = X_test, y_val = y_test, experiment_name = 'multi1', save_weights=False, print_params=True)


  0%|          | 0/16 [00:00<?, ?it/s]

{'num_projection_layers': 0, 'projection_dims': 128, 'dropout_rate': 0.1, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.001, 'epochs': 10, 'batch_size': 32}
52606240/52606240 [==============================] - 0s 0us/step
Epoch 1/10
446/446 [==============================] - 202s 378ms/step - loss: 0.5066 - accuracy: 0.7622 - precision: 0.6845 - recall: 0.5729 - val_loss: 0.5975 - val_accuracy: 0.6875 - val_precision: 0.9128 - val_recall: 0.5551
Epoch 2/10
446/446 [==============================] - 58s 130ms/step - loss: 0.3527 - accuracy: 0.8516 - precision: 0.8133 - recall: 0.7380 - val_loss: 0.4931 - val_accuracy: 0.8565 - val_precision: 0.9175 - val_recall: 0.8476
Epoch 3/10
446/446 [==============================] - 58s 131ms/step - loss: 0.3062 - accuracy: 0.8730 - precision: 0.8384 - recall: 0.7814 - val_loss: 0.4220 - val_accuracy: 0.8492 - val_precision: 0.8707 - val_recall: 0.8923
Epoch 4/10
446/446 [==============================] - 58s 130ms


  6%|▋         | 1/16 [12:16<3:04:08, 736.55s/it]

{'num_projection_layers': 0, 'projection_dims': 128, 'dropout_rate': 0.1, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.001, 'epochs': 10, 'batch_size': 64}
Epoch 1/10
446/446 [==============================] - 105s 199ms/step - loss: 0.5549 - accuracy: 0.7587 - precision: 0.7032 - recall: 0.6252 - val_loss: 0.4312 - val_accuracy: 0.8471 - val_precision: 0.8708 - val_recall: 0.8882
Epoch 2/10
446/446 [==============================] - 59s 131ms/step - loss: 0.3603 - accuracy: 0.8478 - precision: 0.8063 - recall: 0.7339 - val_loss: 0.5696 - val_accuracy: 0.7410 - val_precision: 0.9426 - val_recall: 0.6255
Epoch 3/10
446/446 [==============================] - 58s 130ms/step - loss: 0.3073 - accuracy: 0.8741 - precision: 0.8425 - recall: 0.7798 - val_loss: 0.4728 - val_accuracy: 0.8664 - val_precision: 0.9414 - val_recall: 0.8393
Epoch 4/10
446/446 [==============================] - 58s 131ms/step - loss: 0.2916 - accuracy: 0.8799 - precision: 0.8477 - rec


 12%|█▎        | 2/16 [22:56<2:38:33, 679.55s/it]

...


 88%|████████▊ | 14/16 [2:38:35<22:56, 688.10s/it]

{'num_projection_layers': 0, 'projection_dims': 224, 'dropout_rate': 0.2, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.0005, 'epochs': 10, 'batch_size': 32}
Epoch 1/10
446/446 [==============================] - 108s 206ms/step - loss: 0.5356 - accuracy: 0.7586 - precision: 0.7211 - recall: 0.5885 - val_loss: 0.4686 - val_accuracy: 0.8305 - val_precision: 0.9445 - val_recall: 0.7755
Epoch 2/10
446/446 [==============================] - 63s 142ms/step - loss: 0.3910 - accuracy: 0.8355 - precision: 0.7893 - recall: 0.7119 - val_loss: 0.6264 - val_accuracy: 0.6786 - val_precision: 0.9376 - val_recall: 0.5228
Epoch 3/10
446/446 [==============================] - 63s 141ms/step - loss: 0.3600 - accuracy: 0.8439 - precision: 0.7911 - recall: 0.7421 - val_loss: 0.4784 - val_accuracy: 0.8586 - val_precision: 0.9357 - val_recall: 0.8318
Epoch 4/10
446/446 [==============================] - 63s 142ms/step - loss: 0.3310 - accuracy: 0.8627 - precision: 0.8191 - re


 94%|█████████▍| 15/16 [2:50:05<11:28, 688.56s/it]

{'num_projection_layers': 0, 'projection_dims': 224, 'dropout_rate': 0.2, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.0005, 'epochs': 10, 'batch_size': 64}
Epoch 1/10
446/446 [==============================] - 109s 209ms/step - loss: 0.5480 - accuracy: 0.7493 - precision: 0.7091 - recall: 0.5702 - val_loss: 0.6177 - val_accuracy: 0.7847 - val_precision: 0.9552 - val_recall: 0.6893
Epoch 2/10
446/446 [==============================] - 62s 140ms/step - loss: 0.4026 - accuracy: 0.8256 - precision: 0.7740 - recall: 0.6966 - val_loss: 0.5555 - val_accuracy: 0.7821 - val_precision: 0.9378 - val_recall: 0.6993
Epoch 3/10
446/446 [==============================] - 62s 140ms/step - loss: 0.3682 - accuracy: 0.8455 - precision: 0.7940 - recall: 0.7439 - val_loss: 1.1996 - val_accuracy: 0.3874 - val_precision: 0.6986 - val_recall: 0.0423
Epoch 4/10
446/446 [==============================] - 63s 141ms/step - loss: 0.3833 - accuracy: 0.8455 - precision: 0.7902 - re


100%|██████████| 16/16 [3:02:19<00:00, 683.75s/it]


In [24]:
params = {
    'num_projection_layers' : [1],
    'projection_dims' : [128, 224],
    'dropout_rate' : [0.1, 0.2],
    'vision_trainable' : [False],
    'text_trainable' : [False],
    'attention' : [True],
    'lr' : [0.001, 0.0005],
    'epochs' : [10],
    'batch_size' : [32]
}

In [ ]:
h = ta.Scan(x = X_train, y= y_train, params = params, model = multimodal_model, x_val = X_test, y_val = y_test, experiment_name = 'multi1', save_weights=False, print_params=True)

  0%|          | 0/8 [00:00<?, ?it/s]

{'num_projection_layers': 1, 'projection_dims': 128, 'dropout_rate': 0.1, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.001, 'epochs': 10, 'batch_size': 32}
Epoch 1/10
446/446 [==============================] - 120s 208ms/step - loss: 0.4790 - accuracy: 0.7883 - precision: 0.7164 - recall: 0.6365 - val_loss: 0.4602 - val_accuracy: 0.8346 - val_precision: 0.9397 - val_recall: 0.7871
Epoch 2/10
446/446 [==============================] - 57s 127ms/step - loss: 0.3078 - accuracy: 0.8746 - precision: 0.8324 - recall: 0.7957 - val_loss: 0.5246 - val_accuracy: 0.7400 - val_precision: 0.9164 - val_recall: 0.6446
Epoch 3/10
446/446 [==============================] - 57s 129ms/step - loss: 0.2658 - accuracy: 0.8926 - precision: 0.8580 - recall: 0.8241 - val_loss: 0.4445 - val_accuracy: 0.8383 - val_precision: 0.8972 - val_recall: 0.8384
Epoch 4/10
446/446 [==============================] - ETA: 0s - loss: 0.2417 - accuracy: 0.9045 - precision: 0.8772 - recall: 0.